In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
# from ucimlrepo import fetch_ucirepo 

In [25]:
# full_data=pd.concat([X2,y], axis=1)
# values_to_match=['ARP_poisioning', 'DDOS_Slowloris', 'DOS_SYN_Hping',
#                  'Metasploit_Brute_Force_SSH', 'NMAP_FIN_SCAN',
#                  'NMAP_OS_DETECTION', 'NMAP_TCP_scan', 'NMAP_UDP_SCAN',
#                  'NMAP_XMAS_TREE_SCAN']
# anomalous_data=full_data[full_data['Attack_type'].isin(values_to_match)]
# anomalous_data=anomalous_data.iloc[:, :-1]
# # anomalous_data
# print(anomalous_data[anomalous_data.notna().any(axis=1)].shape, anomalous_data.shape)
# anomalous_data

In [7]:
from datasets import load_dataset
dataset = load_dataset("aamui/rt_iot_2022")

Generating train split: 0 examples [00:00, ? examples/s]

In [40]:
dataset_full=dataset['train'].to_pandas().filter(regex='^((?!max|min|avg|std).)*$')
targets=dataset_full['Attack_type']
X=dataset_full.drop('Attack_type', axis=1)

# droped id cols + proto, service
X=X.drop(X.columns[[0,1,2,3]], axis=1)

In [43]:
print(X.dtypes)

flow_duration               float64
fwd_pkts_tot                  int64
bwd_pkts_tot                  int64
fwd_data_pkts_tot             int64
bwd_data_pkts_tot             int64
fwd_pkts_per_sec            float64
bwd_pkts_per_sec            float64
flow_pkts_per_sec           float64
down_up_ratio               float64
fwd_header_size_tot           int64
bwd_header_size_tot           int64
flow_FIN_flag_count           int64
flow_SYN_flag_count           int64
flow_RST_flag_count           int64
fwd_PSH_flag_count            int64
bwd_PSH_flag_count            int64
flow_ACK_flag_count           int64
fwd_URG_flag_count            int64
bwd_URG_flag_count            int64
flow_CWR_flag_count           int64
flow_ECE_flag_count           int64
fwd_pkts_payload.tot        float64
bwd_pkts_payload.tot        float64
flow_pkts_payload.tot       float64
fwd_iat.tot                 float64
bwd_iat.tot                 float64
flow_iat.tot                float64
payload_bytes_per_second    

In [45]:
def beta_elbow(x_hat, x, beta, N, z_mu, z_sigma):
    # this loss assumes that all columns are continuous which i believe is true
    beta_cross_entropy=-((beta+1)/beta) * torch.mean(torch.exp((-0.5 * z_sigma * beta) * torch.sum((x_hat - x)**2))-1)
    kl_div=torch.mean(0.5 * torch.sum(1 + 2 * torch.log(z_sigma) - (z_mu**2)  - (z_sigma**2), -1))
    return beta_cross_entropy + kl_div